In [1]:
import math
import numpy as np
import k_armed_bandit as lib

Parameters

In [2]:
k = 10
q_real = [0, 4, 2, 9, 1, 3, 8, 6, 7, 5]
iteration_no = 10000
epsilon = 0.1
standard_deviation = 1

### Epsilon greedy action value method

In [3]:
Q = [0] * k
N = [0] * k
cumulative_reward = 0   # just for statistics purpose
for i in range(iteration_no):
    probability = np.random.rand()
    if probability > epsilon:
        action = lib.get_greedy_action(Q)
    else:
        action = np.random.randint(k)

    reward = lib.get_action_reward(q_real[action], standard_deviation)
    N[action] += 1
    Q[action] += (reward - Q[action]) / N[action]
    cumulative_reward += reward
lib.print_stats(
    q_real=q_real,
    N=N,
    iteration_no=iteration_no,
    cumulative_reward=cumulative_reward,
    Q=Q,
    epsilon=epsilon
)

Percentage of taking best posible action in 10000 itearation: 90.34 %, additional_factor:  0.10

Average reward:     8.53

Q[0] =  0.07	q_real[0] =  0.00	N[0] = 114.00
Q[1] =  3.96	q_real[1] =  4.00	N[1] = 104.00
Q[2] =  2.00	q_real[2] =  2.00	N[2] = 108.00
Q[3] =  9.00	q_real[3] =  9.00	N[3] = 9034.00
Q[4] =  1.12	q_real[4] =  1.00	N[4] = 91.00
Q[5] =  3.06	q_real[5] =  3.00	N[5] = 98.00
Q[6] =  8.07	q_real[6] =  8.00	N[6] = 104.00
Q[7] =  5.98	q_real[7] =  6.00	N[7] = 135.00
Q[8] =  7.05	q_real[8] =  7.00	N[8] = 89.00
Q[9] =  5.01	q_real[9] =  5.00	N[9] = 123.00


### Optimistic initial values

In [4]:
init_value = 10
Q = [init_value] * k
N = [0] * k
cumulative_reward = 0   # just for statistics purpose
for i in range(iteration_no):
    action = lib.get_greedy_action(Q)
    reward = lib.get_action_reward(q_real[action], standard_deviation)
    N[action] += 1
    Q[action] += (reward - Q[action]) / N[action]
    cumulative_reward += reward
lib.print_stats(
    q_real=q_real,
    N=N,
    iteration_no=iteration_no,
    cumulative_reward=cumulative_reward,
    Q=Q,
    init_value=init_value)

Percentage of taking best posible action in 10000 itearation: 99.91 %, additional_factor: 10.00

Average reward:     8.99

Q[0] = -0.39	q_real[0] =  0.00	N[0] =  1.00
Q[1] =  4.06	q_real[1] =  4.00	N[1] =  1.00
Q[2] = -0.80	q_real[2] =  2.00	N[2] =  1.00
Q[3] =  8.99	q_real[3] =  9.00	N[3] = 9991.00
Q[4] =  2.66	q_real[4] =  1.00	N[4] =  1.00
Q[5] =  3.17	q_real[5] =  3.00	N[5] =  1.00
Q[6] =  8.91	q_real[6] =  8.00	N[6] =  1.00
Q[7] =  5.23	q_real[7] =  6.00	N[7] =  1.00
Q[8] =  6.18	q_real[8] =  7.00	N[8] =  1.00
Q[9] =  5.87	q_real[9] =  5.00	N[9] =  1.00


###  Upper-Confidence-Bound Action Selection

In [5]:
c = 1
Q = [0] * k
N = np.zeros(k)
cumulative_reward = 0   # just for statistics purpose

#first loop overe each action is neccery to avoid division by N[i] == 0
for i in range(k):
    reward = lib.get_action_reward(q_real[i], standard_deviation)
    N[i] += 1
    Q[i] += (reward - Q[i]) / N[i]
    cumulative_reward += reward
    
    
for i in range(k, iteration_no):
    UCB = Q + c * np.sqrt(math.log(i) / N)
    action = lib.get_greedy_action(UCB)
    reward = lib.get_action_reward(q_real[action], standard_deviation)
    N[action] += 1
    Q[action] += (reward - Q[action]) / N[action]
    cumulative_reward += reward

lib.print_stats(
    q_real=q_real,
    N=N,
    iteration_no=iteration_no,
    cumulative_reward=cumulative_reward,
    Q=Q,
    c=c
)

Percentage of taking best posible action in 10000 itearation: 99.81 %, additional_factor:  1.00

Average reward:     8.98

Q[0] = -1.15	q_real[0] =  0.00	N[0] =  1.00
Q[1] =  3.55	q_real[1] =  4.00	N[1] =  1.00
Q[2] =  2.94	q_real[2] =  2.00	N[2] =  1.00
Q[3] =  8.99	q_real[3] =  9.00	N[3] = 9981.00
Q[4] = -0.08	q_real[4] =  1.00	N[4] =  1.00
Q[5] =  4.08	q_real[5] =  3.00	N[5] =  1.00
Q[6] =  7.79	q_real[6] =  8.00	N[6] =  7.00
Q[7] =  6.43	q_real[7] =  6.00	N[7] =  2.00
Q[8] =  6.54	q_real[8] =  7.00	N[8] =  3.00
Q[9] =  5.64	q_real[9] =  5.00	N[9] =  2.00


### Gradient Bandit Algorithm

In [7]:
#Unstable due to numerical errors
alpha = 0.25
H = np.zeros(k)
N = np.zeros(k)   # just for statistics purpose
cumulative_reward = 0   # just for statistics purpose

policy = np.full(k, 1/k)
reward_mean = 0

def get_action_gradient_bandit_algorithm(policy: np.array, probability: float):
    for i, numerical_preference in enumerate(policy):
        if probability < numerical_preference:
            return i
        else:
            probability -= numerical_preference
    print(probability, np.sum(policy), policy)
    raise Exception("Numerical error")
    
for i in range(iteration_no):
    probability = np.random.rand()
    action = get_action_gradient_bandit_algorithm(policy, probability)
    reward = lib.get_action_reward(q_real[action], standard_deviation)
    reward_mean += (reward - reward_mean)/(i + 1)
    for j in range(k):
        if j == action:
            H[j] += alpha * (reward - reward_mean) * (1.0 - policy[j])
        else:
            H[j] += alpha * (reward - reward_mean) * policy[j]

    policy = np.exp(H) / np.sum(np.exp(H))
    N[action] += 1
    cumulative_reward += reward
    
lib.print_stats(
    q_real=q_real,
    N=N,
    iteration_no=iteration_no,
    cumulative_reward=cumulative_reward,
    policy=policy,
    H=H,
    alpha=alpha)

Percentage of taking best posible action in 10000 itearation: 76.82 %, additional_factor:  0.25

Average reward:     8.50

H[0] = -117.22	q_real[0] =  0.00	policy[0] =  0.00	N[0] =    53
H[1] = -117.35	q_real[1] =  4.00	policy[1] =  0.00	N[1] =    88
H[2] = -116.86	q_real[2] =  2.00	policy[2] =  0.00	N[2] =    64
H[3] = -16.39	q_real[3] =  9.00	policy[3] =  0.86	N[3] =  7682
H[4] = -118.10	q_real[4] =  1.00	policy[4] =  0.00	N[4] =    55
H[5] = -117.16	q_real[5] =  3.00	policy[5] =  0.00	N[5] =    79
H[6] = -18.25	q_real[6] =  8.00	policy[6] =  0.14	N[6] =  1534
H[7] = -116.52	q_real[7] =  6.00	policy[7] =  0.00	N[7] =   142
H[8] = -116.51	q_real[8] =  7.00	policy[8] =  0.00	N[8] =   191
H[9] = -116.49	q_real[9] =  5.00	policy[9] =  0.00	N[9] =   112
